# Model Training
Phase 4: 모델 학습 & 최적화

**목표:**
- Train/Test 분할 (시간 기반)
- 클래스 불균형 처리 (SMOTE)
- 점진적 모델 성능 개선 (단순 → 복잡)
- 최적 모델 선택 & 하이퍼파라미터 튜닝
- 최종 모델 저장

**모델 학습 순서:** Logistic Regression → Decision Tree → Random Forest → Gradient Boosting → XGBoost

---
## 1. 환경 설정

In [1]:
import sys
import os
import json

# === 경로 설정 ===
# 데이터 경로 (03에서 생성한 파일)
data_dir = '/Users/kuka/5X Finder/data/processed'

# 모델 저장 경로
model_dir = '/Users/kuka/5X Finder/models'
os.makedirs(model_dir, exist_ok=True)

print(f"데이터 경로: {data_dir}")
print(f"모델 저장 경로: {model_dir}")

데이터 경로: /Users/kuka/5X Finder/data/processed
모델 저장 경로: /Users/kuka/5X Finder/models


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 시각화
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

# ML 라이브러리
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)

# 클래스 불균형 처리
from imblearn.over_sampling import SMOTE

import joblib

print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


---
## 2. 데이터 로드

03에서 저장한 `ml_dataset.parquet` 로드

In [3]:
# 데이터셋 로드
dataset = pd.read_parquet(os.path.join(data_dir, 'ml_dataset.parquet'))

# Feature 목록 로드
with open(os.path.join(data_dir, 'feature_columns.txt'), 'r') as f:
    feature_cols = [line.strip() for line in f.readlines()]

print(f"=== 데이터셋 로드 완료 ===")
print(f"Shape: {dataset.shape}")
print(f"Feature 수: {len(feature_cols)}개")
print(f"기간: {dataset['start_year'].min()} - {dataset['start_year'].max()}")

=== 데이터셋 로드 완료 ===
Shape: (4652, 30)
Feature 수: 26개
기간: 2010 - 2019


In [4]:
# Target 분포 확인
target_dist = dataset['target_5x'].value_counts()
target_pct = dataset['target_5x'].value_counts(normalize=True) * 100

print(f"\n=== Target 분포 ===")
print(f"미달성 (0): {target_dist[0]:,}개 ({target_pct[0]:.1f}%)")
print(f"5배 달성 (1): {target_dist[1]:,}개 ({target_pct[1]:.1f}%)")


=== Target 분포 ===
미달성 (0): 4,439개 (95.4%)
5배 달성 (1): 213개 (4.6%)


---
## 3. Train/Test 분할

### 왜 시간 기반 분할?
- 랜덤 분할하면 2019년 데이터로 2010년 예측하는 꼴
- 실제 투자에서는 **과거 데이터로 미래 예측**
- Train: 2010-2017 (8년), Test: 2018-2019 (2년)

In [5]:
# 시간 기반 분할
train_years = list(range(2010, 2018))  # 2010-2017
test_years = list(range(2018, 2020))   # 2018-2019

train_data = dataset[dataset['start_year'].isin(train_years)]
test_data = dataset[dataset['start_year'].isin(test_years)]

# Feature / Target 분리
X_train = train_data[feature_cols]
y_train = train_data['target_5x']

X_test = test_data[feature_cols]
y_test = test_data['target_5x']

print(f"=== Train/Test 분할 ===")
print(f"Train: {len(X_train):,}개 ({train_years[0]}-{train_years[-1]})")
print(f"Test: {len(X_test):,}개 ({test_years[0]}-{test_years[-1]})")

=== Train/Test 분할 ===
Train: 3,681개 (2010-2017)
Test: 971개 (2018-2019)


In [6]:
# Train/Test Target 분포 확인
print(f"\n=== Train Target 분포 ===")
print(f"미달성: {(y_train == 0).sum():,}개 ({100*(y_train == 0).mean():.1f}%)")
print(f"5배 달성: {(y_train == 1).sum():,}개 ({100*(y_train == 1).mean():.1f}%)")

print(f"\n=== Test Target 분포 ===")
print(f"미달성: {(y_test == 0).sum():,}개 ({100*(y_test == 0).mean():.1f}%)")
print(f"5배 달성: {(y_test == 1).sum():,}개 ({100*(y_test == 1).mean():.1f}%)")


=== Train Target 분포 ===
미달성: 3,499개 (95.1%)
5배 달성: 182개 (4.9%)

=== Test Target 분포 ===
미달성: 940개 (96.8%)
5배 달성: 31개 (3.2%)


In [7]:
# 분할 시각화
yearly_split = []
for year in range(2010, 2020):
    year_data = dataset[dataset['start_year'] == year]
    yearly_split.append({
        'year': year,
        'count': len(year_data),
        'split': 'Train' if year in train_years else 'Test'
    })

split_df = pd.DataFrame(yearly_split)

fig = px.bar(
    split_df, x='year', y='count', color='split',
    color_discrete_map={'Train': '#3498db', 'Test': '#e74c3c'},
    title='연도별 Train/Test 분할',
    labels={'year': '연도', 'count': '샘플 수', 'split': '구분'},
    text='count'
)
fig.update_traces(textposition='outside')
fig.update_layout(height=400)
fig.show()

---
## 4. 클래스 불균형 처리

### 왜 SMOTE?
- 5배 달성 종목이 **4.6%**밖에 안 됨
- 그냥 학습하면 모델이 "다 0이야" 라고 예측해버림
- SMOTE로 소수 클래스 오버샘플링

**주의:** Train 데이터에만 적용! (Test는 원본 유지)

In [8]:
# SMOTE 적용 전 분포
print(f"=== SMOTE 적용 전 (Train) ===")
print(f"미달성: {(y_train == 0).sum():,}개")
print(f"5배 달성: {(y_train == 1).sum():,}개")
print(f"비율: 1 : {(y_train == 0).sum() / (y_train == 1).sum():.1f}")

=== SMOTE 적용 전 (Train) ===
미달성: 3,499개
5배 달성: 182개
비율: 1 : 19.2


In [9]:
# SMOTE 적용
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"\n=== SMOTE 적용 후 (Train) ===")
print(f"미달성: {(y_train_resampled == 0).sum():,}개")
print(f"5배 달성: {(y_train_resampled == 1).sum():,}개")
print(f"비율: 1 : {(y_train_resampled == 0).sum() / (y_train_resampled == 1).sum():.1f}")


=== SMOTE 적용 후 (Train) ===
미달성: 3,499개
5배 달성: 3,499개
비율: 1 : 1.0


In [10]:
# SMOTE 전/후 비교 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['SMOTE 적용 전', 'SMOTE 적용 후'])

# 적용 전
fig.add_trace(go.Bar(
    x=['미달성', '5배 달성'],
    y=[(y_train == 0).sum(), (y_train == 1).sum()],
    marker_color=['#3498db', '#e74c3c'],
    text=[(y_train == 0).sum(), (y_train == 1).sum()],
    textposition='outside'
), row=1, col=1)

# 적용 후
fig.add_trace(go.Bar(
    x=['미달성', '5배 달성'],
    y=[(y_train_resampled == 0).sum(), (y_train_resampled == 1).sum()],
    marker_color=['#3498db', '#e74c3c'],
    text=[(y_train_resampled == 0).sum(), (y_train_resampled == 1).sum()],
    textposition='outside'
), row=1, col=2)

fig.update_layout(height=400, showlegend=False, title='SMOTE 적용 전/후 클래스 분포')
fig.show()

---
## 5. 점진적 모델 학습 & 비교

단순한 모델부터 시작해서 점차 복잡한 모델로 성능을 개선해나갑니다.

In [11]:
# 모델 평가 함수 정의
def evaluate_model(model, X_test, y_test, model_name):
    """모델 평가 및 결과 반환"""
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    results = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0),
        'F1': f1_score(y_test, y_pred, zero_division=0),
        'ROC-AUC': roc_auc_score(y_test, y_pred_proba)
    }
    return results, y_pred, y_pred_proba

# 결과 저장 리스트
all_results = []
trained_models = {}

print("✅ 평가 함수 정의 완료")

✅ 평가 함수 정의 완료


### 5-1. Logistic Regression (베이스라인)

가장 간단한 모델로 **베이스라인** 성능을 설정합니다.
- 선형 모델: Feature와 Target의 선형 관계만 학습
- 해석이 쉬움: 각 Feature의 영향력을 계수로 확인 가능
- 이 성능이 **최소 기준점**이 됨

In [12]:
# 5-1. Logistic Regression
print("=" * 50)
print("5-1. Logistic Regression (베이스라인)")
print("=" * 50)

model_lr = LogisticRegression(max_iter=1000, random_state=42)
model_lr.fit(X_train_resampled, y_train_resampled)

results_lr, y_pred_lr, y_proba_lr = evaluate_model(model_lr, X_test, y_test, 'Logistic Regression')
all_results.append(results_lr)
trained_models['Logistic Regression'] = model_lr

print(f"\n📊 베이스라인 성능")
print(f"   ROC-AUC: {results_lr['ROC-AUC']:.4f}")
print(f"   Recall: {results_lr['Recall']:.4f}")
print(f"   Precision: {results_lr['Precision']:.4f}")

5-1. Logistic Regression (베이스라인)



📊 베이스라인 성능
   ROC-AUC: 0.8716
   Recall: 0.8065
   Precision: 0.1106


### 5-2. Decision Tree (트리 모델)

비선형 관계를 학습할 수 있는 **트리 모델**입니다.
- 비선형 모델: Feature 간 복잡한 상호작용 학습 가능
- 단점: 과적합 위험이 높음
- 베이스라인 대비 개선되는지 확인

In [13]:
# 5-2. Decision Tree
print("=" * 50)
print("5-2. Decision Tree")
print("=" * 50)

model_dt = DecisionTreeClassifier(max_depth=10, random_state=42)
model_dt.fit(X_train_resampled, y_train_resampled)

results_dt, y_pred_dt, y_proba_dt = evaluate_model(model_dt, X_test, y_test, 'Decision Tree')
all_results.append(results_dt)
trained_models['Decision Tree'] = model_dt

# 베이스라인 대비 비교
auc_diff = results_dt['ROC-AUC'] - results_lr['ROC-AUC']
recall_diff = results_dt['Recall'] - results_lr['Recall']

print(f"\n📊 성능")
print(f"   ROC-AUC: {results_dt['ROC-AUC']:.4f} ({'+' if auc_diff >= 0 else ''}{auc_diff:.4f})")
print(f"   Recall: {results_dt['Recall']:.4f} ({'+' if recall_diff >= 0 else ''}{recall_diff:.4f})")
print(f"   Precision: {results_dt['Precision']:.4f}")

# 성능 판정
if auc_diff > 0.01:
    verdict = "개선 ✅"
elif auc_diff < -0.01:
    verdict = "하락 ❌"
else:
    verdict = "유사한 수준"
print(f"\n→ 베이스라인 대비 {verdict}")

5-2. Decision Tree

📊 성능
   ROC-AUC: 0.8730 (+0.0014)
   Recall: 0.7742 (-0.0323)
   Precision: 0.1714

→ 베이스라인 대비 유사한 수준


### 5-3. Random Forest (앙상블 - 배깅)

트리 여러 개를 **병렬로** 학습합니다 (배깅).
- 여러 Decision Tree의 평균 → 과적합 감소
- 각 트리는 데이터의 일부만 사용 (Bootstrap)
- 단일 트리보다 안정적인 성능

In [14]:
# 5-3. Random Forest
print("=" * 50)
print("5-3. Random Forest (앙상블 - 배깅)")
print("=" * 50)

model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_rf.fit(X_train_resampled, y_train_resampled)

results_rf, y_pred_rf, y_proba_rf = evaluate_model(model_rf, X_test, y_test, 'Random Forest')
all_results.append(results_rf)
trained_models['Random Forest'] = model_rf

# 이전 모델 대비 비교 (Decision Tree)
auc_diff = results_rf['ROC-AUC'] - results_dt['ROC-AUC']
recall_diff = results_rf['Recall'] - results_dt['Recall']

print(f"\n📊 성능")
print(f"   ROC-AUC: {results_rf['ROC-AUC']:.4f} (vs DT: {'+' if auc_diff >= 0 else ''}{auc_diff:.4f})")
print(f"   Recall: {results_rf['Recall']:.4f} (vs DT: {'+' if recall_diff >= 0 else ''}{recall_diff:.4f})")
print(f"   Precision: {results_rf['Precision']:.4f}")

if auc_diff > 0.01:
    verdict = "개선 ✅"
elif auc_diff < -0.01:
    verdict = "하락 ❌"
else:
    verdict = "유사한 수준"
print(f"\n→ 트리 100개 앙상블로 {verdict}")

5-3. Random Forest (앙상블 - 배깅)

📊 성능
   ROC-AUC: 0.8817 (vs DT: +0.0088)
   Recall: 0.6774 (vs DT: -0.0968)
   Precision: 0.1694

→ 트리 100개 앙상블로 유사한 수준


### 5-4. Gradient Boosting (앙상블 - 부스팅)

이전 모델의 오차를 **순차적으로** 학습합니다 (부스팅).
- 배깅 (RF): 트리들이 독립적으로 학습
- 부스팅 (GB): 이전 트리의 실수를 다음 트리가 보완
- 일반적으로 배깅보다 성능이 좋지만, 과적합 위험 있음

In [15]:
# 5-4. Gradient Boosting
print("=" * 50)
print("5-4. Gradient Boosting (앙상블 - 부스팅)")
print("=" * 50)

model_gb = GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42)
model_gb.fit(X_train_resampled, y_train_resampled)

results_gb, y_pred_gb, y_proba_gb = evaluate_model(model_gb, X_test, y_test, 'Gradient Boosting')
all_results.append(results_gb)
trained_models['Gradient Boosting'] = model_gb

# 이전 모델 대비 비교 (Random Forest)
auc_diff = results_gb['ROC-AUC'] - results_rf['ROC-AUC']
recall_diff = results_gb['Recall'] - results_rf['Recall']

print(f"\n📊 성능")
print(f"   ROC-AUC: {results_gb['ROC-AUC']:.4f} (vs RF: {'+' if auc_diff >= 0 else ''}{auc_diff:.4f})")
print(f"   Recall: {results_gb['Recall']:.4f} (vs RF: {'+' if recall_diff >= 0 else ''}{recall_diff:.4f})")
print(f"   Precision: {results_gb['Precision']:.4f}")

if auc_diff > 0.01:
    verdict = "개선 ✅"
elif auc_diff < -0.01:
    verdict = "하락 ❌"
else:
    verdict = "유사한 수준"
print(f"\n→ 부스팅 방식으로 {verdict}")

5-4. Gradient Boosting (앙상블 - 부스팅)

📊 성능
   ROC-AUC: 0.8865 (vs RF: +0.0048)
   Recall: 0.6452 (vs RF: -0.0323)
   Precision: 0.1802

→ 부스팅 방식으로 유사한 수준


### 5-5. XGBoost (개선된 부스팅)

Gradient Boosting의 **개선된 버전**입니다.
- 정규화 추가: 과적합 방지
- 병렬 처리: 학습 속도 향상
- 결측치 자동 처리
- 캐글 등 대회에서 가장 많이 사용되는 모델

In [16]:
# 5-5. XGBoost
print("=" * 50)
print("5-5. XGBoost (개선된 부스팅)")
print("=" * 50)

model_xgb = XGBClassifier(n_estimators=100, max_depth=5, random_state=42, eval_metric='logloss')
model_xgb.fit(X_train_resampled, y_train_resampled)

results_xgb, y_pred_xgb, y_proba_xgb = evaluate_model(model_xgb, X_test, y_test, 'XGBoost')
all_results.append(results_xgb)
trained_models['XGBoost'] = model_xgb

# 이전 모델 대비 비교 (Gradient Boosting)
auc_diff = results_xgb['ROC-AUC'] - results_gb['ROC-AUC']
recall_diff = results_xgb['Recall'] - results_gb['Recall']

print(f"\n📊 성능")
print(f"   ROC-AUC: {results_xgb['ROC-AUC']:.4f} (vs GB: {'+' if auc_diff >= 0 else ''}{auc_diff:.4f})")
print(f"   Recall: {results_xgb['Recall']:.4f} (vs GB: {'+' if recall_diff >= 0 else ''}{recall_diff:.4f})")
print(f"   Precision: {results_xgb['Precision']:.4f}")

if auc_diff > 0.01:
    verdict = "개선 ✅"
elif auc_diff < -0.01:
    verdict = "하락 ❌"
else:
    verdict = "유사한 수준"
print(f"\n→ 개선된 부스팅으로 {verdict}")

5-5. XGBoost (개선된 부스팅)

📊 성능
   ROC-AUC: 0.8778 (vs GB: -0.0088)
   Recall: 0.6452 (vs GB: +0.0000)
   Precision: 0.1961

→ 개선된 부스팅으로 유사한 수준


### 5-6. 전체 모델 비교

In [17]:
# 결과 비교 테이블
results_df = pd.DataFrame(all_results)

# 학습 순서 유지
model_order = ['Logistic Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting', 'XGBoost']
results_df['order'] = results_df['Model'].map({m: i for i, m in enumerate(model_order)})
results_df = results_df.sort_values('order').drop('order', axis=1).reset_index(drop=True)

print("=== 전체 모델 성능 비교 (학습 순서) ===")
print(results_df.to_string(index=False))

=== 전체 모델 성능 비교 (학습 순서) ===
              Model  Accuracy  Precision   Recall       F1  ROC-AUC
Logistic Regression  0.786818   0.110619 0.806452 0.194553 0.871568
      Decision Tree  0.873326   0.171429 0.774194 0.280702 0.872975
      Random Forest  0.883625   0.169355 0.677419 0.270968 0.881726
  Gradient Boosting  0.894954   0.180180 0.645161 0.281690 0.886531
            XGBoost  0.904222   0.196078 0.645161 0.300752 0.877780


In [18]:
# 점진적 개선 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['ROC-AUC 변화', 'Recall 변화'])

# ROC-AUC
fig.add_trace(go.Scatter(
    x=results_df['Model'],
    y=results_df['ROC-AUC'],
    mode='lines+markers+text',
    marker=dict(size=12, color='#3498db'),
    line=dict(width=2, color='#3498db'),
    text=results_df['ROC-AUC'].round(3),
    textposition='top center',
    name='ROC-AUC'
), row=1, col=1)

# Recall
fig.add_trace(go.Scatter(
    x=results_df['Model'],
    y=results_df['Recall'],
    mode='lines+markers+text',
    marker=dict(size=12, color='#e74c3c'),
    line=dict(width=2, color='#e74c3c'),
    text=results_df['Recall'].round(3),
    textposition='top center',
    name='Recall'
), row=1, col=2)

fig.update_layout(
    height=450,
    showlegend=False,
    title='점진적 모델 성능 변화 (단순 → 복잡)'
)
fig.update_xaxes(tickangle=45)
fig.show()

In [34]:
# 베이스라인 대비 개선율
baseline_auc = results_df[results_df['Model'] == 'Logistic Regression']['ROC-AUC'].values[0]
baseline_recall = results_df[results_df['Model'] == 'Logistic Regression']['Recall'].values[0]

print("=== 베이스라인 (Logistic Regression) 대비 개선 ===")
print(f"{'모델':<20} {'ROC-AUC 변화':>15} {'Recall 변화':>15}")
print("-" * 55)

for _, row in results_df.iterrows():
    if row['Model'] == 'Logistic Regression':  # 베이스라인 스킵
        continue
    auc_diff = row['ROC-AUC'] - baseline_auc
    recall_diff = row['Recall'] - baseline_recall
    auc_str = f"{'+' if auc_diff >= 0 else ''}{auc_diff:.4f}"
    recall_str = f"{'+' if recall_diff >= 0 else ''}{recall_diff:.4f}"
    print(f"{row['Model']:<20} {auc_str:>15} {recall_str:>15}")

=== 베이스라인 (Logistic Regression) 대비 개선 ===
모델                        ROC-AUC 변화       Recall 변화
-------------------------------------------------------
Decision Tree                +0.0014         -0.0323
Random Forest                +0.0102         -0.1290
Gradient Boosting            +0.0150         -0.1613
XGBoost                      +0.0062         -0.1613


---
## 6. 최적 모델 선택

### 선택 기준
- **ROC-AUC**: 전체 분류 성능
- **Recall**: 5배 종목을 놓치지 않는 것이 중요 (투자 관점)

In [39]:
# ROC-AUC 기준 최적 모델
best_by_auc = results_df.loc[results_df['ROC-AUC'].idxmax()]

# Recall 기준 최적 모델
best_by_recall = results_df.loc[results_df['Recall'].idxmax()]

print("=== 최적 모델 선택 ===")
print(f"\n📊 ROC-AUC 기준 최적: {best_by_auc['Model']}")
print(f"   ROC-AUC: {best_by_auc['ROC-AUC']:.4f}")
print(f"   Recall: {best_by_auc['Recall']:.4f}")

print(f"\n📊 Recall 기준 최적: {best_by_recall['Model']}")
print(f"   Recall: {best_by_recall['Recall']:.4f}")
print(f"   ROC-AUC: {best_by_recall['ROC-AUC']:.4f}")

=== 최적 모델 선택 ===

📊 ROC-AUC 기준 최적: Gradient Boosting
   ROC-AUC: 0.8865
   Recall: 0.6452

📊 Recall 기준 최적: Logistic Regression
   Recall: 0.8065
   ROC-AUC: 0.8716


In [40]:
# 최종 선택: Recall 기준
best_model_name = best_by_recall['Model']
best_model = trained_models[best_model_name]

print(f"\n✅ 최종 선택: {best_model_name}")
print(f"   → Recall 기준 (5배 종목을 놓치지 않는 것이 중요)")


✅ 최종 선택: Logistic Regression
   → Recall 기준 (5배 종목을 놓치지 않는 것이 중요)


---
## 7. 하이퍼파라미터 튜닝

### GridSearchCV
- 교차 검증으로 과적합 방지
- 최적 파라미터 탐색

In [41]:
# 최적 모델별 하이퍼파라미터 그리드 정의
param_grids = {
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10],
        'penalty': ['l2'],
        'solver': ['lbfgs']
    },
    'XGBoost': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15],
        'min_samples_split': [2, 5, 10]
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'Decision Tree': {
        'max_depth': [5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
}

print(f"튜닝할 모델: {best_model_name}")
print(f"파라미터 그리드: {param_grids[best_model_name]}")

튜닝할 모델: Logistic Regression
파라미터 그리드: {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs']}


In [46]:
# GridSearchCV 실행
print(f"GridSearchCV 실행 중... (시간이 걸릴 수 있습니다)")

# 모델 재생성 (튜닝용)
if best_model_name == 'Logistic Regression':
    base_model = LogisticRegression(max_iter=1000, random_state=42)
elif best_model_name == 'XGBoost':
    base_model = XGBClassifier(random_state=42, eval_metric='logloss')
elif best_model_name == 'Random Forest':
    base_model = RandomForestClassifier(random_state=42)
elif best_model_name == 'Gradient Boosting':
    base_model = GradientBoostingClassifier(random_state=42)
else:
    base_model = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(
    base_model,
    param_grids[best_model_name],
    cv=5,
    scoring='roc_auc',
    n_jobs=1,
    verbose=1
)

grid_search.fit(X_train_resampled, y_train_resampled)

print(f"\n✅ GridSearchCV 완료")
print(f"최적 파라미터: {grid_search.best_params_}")
print(f"최적 CV Score: {grid_search.best_score_:.4f}")

GridSearchCV 실행 중... (시간이 걸릴 수 있습니다)
Fitting 5 folds for each of 4 candidates, totalling 20 fits

✅ GridSearchCV 완료
최적 파라미터: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
최적 CV Score: 0.8822


In [47]:
# 튜닝 전/후 비교
tuned_model = grid_search.best_estimator_

# 튜닝 전 (기존 모델)
y_pred_before = best_model.predict(X_test)
y_pred_proba_before = best_model.predict_proba(X_test)[:, 1]

# 튜닝 후
y_pred_after = tuned_model.predict(X_test)
y_pred_proba_after = tuned_model.predict_proba(X_test)[:, 1]

print(f"=== 튜닝 전/후 비교 ===")
print(f"{'메트릭':<15} {'튜닝 전':>10} {'튜닝 후':>10} {'변화':>10}")
print("-" * 50)

metrics = [
    ('ROC-AUC', roc_auc_score(y_test, y_pred_proba_before), roc_auc_score(y_test, y_pred_proba_after)),
    ('Recall', recall_score(y_test, y_pred_before), recall_score(y_test, y_pred_after)),
    ('Precision', precision_score(y_test, y_pred_before, zero_division=0), precision_score(y_test, y_pred_after, zero_division=0)),
    ('F1', f1_score(y_test, y_pred_before), f1_score(y_test, y_pred_after))
]

for name, before, after in metrics:
    change = after - before
    sign = '+' if change >= 0 else ''
    print(f"{name:<15} {before:>10.4f} {after:>10.4f} {sign}{change:>9.4f}")

=== 튜닝 전/후 비교 ===
메트릭                   튜닝 전       튜닝 후         변화
--------------------------------------------------
ROC-AUC             0.8716     0.8716 +   0.0000
Recall              0.8065     0.8065 +   0.0000
Precision           0.1106     0.1106 +   0.0000
F1                  0.1946     0.1946 +   0.0000


---
## 8. 최종 모델 평가

In [45]:
# 최종 모델 설정
final_model = tuned_model

# 최종 예측
y_final_pred = final_model.predict(X_test)
y_final_proba = final_model.predict_proba(X_test)[:, 1]

print(f"=== 최종 모델: {best_model_name} (튜닝됨) ===")

=== 최종 모델: Logistic Regression (튜닝됨) ===


In [48]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_final_pred)

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['예측: 미달성', '예측: 5배 달성'],
    y=['실제: 미달성', '실제: 5배 달성'],
    text=cm,
    texttemplate='%{text}',
    textfont={'size': 20},
    colorscale='Blues'
))

fig.update_layout(
    title='Confusion Matrix',
    height=400,
    width=500
)
fig.show()

# 해석
tn, fp, fn, tp = cm.ravel()
print(f"\n=== Confusion Matrix 해석 ===")
print(f"True Negative (정확히 미달성 예측): {tn}")
print(f"False Positive (미달성인데 5배 예측): {fp}")
print(f"False Negative (5배인데 미달성 예측): {fn}")
print(f"True Positive (정확히 5배 예측): {tp}")


=== Confusion Matrix 해석 ===
True Negative (정확히 미달성 예측): 739
False Positive (미달성인데 5배 예측): 201
False Negative (5배인데 미달성 예측): 6
True Positive (정확히 5배 예측): 25


In [49]:
# Classification Report
print("=== Classification Report ===")
print(classification_report(y_test, y_final_pred, target_names=['미달성', '5배 달성']))

=== Classification Report ===
              precision    recall  f1-score   support

         미달성       0.99      0.79      0.88       940
       5배 달성       0.11      0.81      0.19        31

    accuracy                           0.79       971
   macro avg       0.55      0.80      0.54       971
weighted avg       0.96      0.79      0.86       971



In [50]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_final_proba)
roc_auc = roc_auc_score(y_test, y_final_proba)

fig = go.Figure()

# ROC Curve
fig.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.3f})',
    line=dict(color='#e74c3c', width=2)
))

# 랜덤 기준선
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random (AUC = 0.5)',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title='ROC Curve',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=500,
    width=600
)
fig.show()

In [51]:
# Feature Importance (트리 기반 모델인 경우)
if hasattr(final_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=True).tail(15)
    
    fig = px.bar(
        importance_df,
        x='importance',
        y='feature',
        orientation='h',
        title='Feature Importance (상위 15개)',
        color='importance',
        color_continuous_scale='Reds'
    )
    fig.update_layout(height=500, showlegend=False)
    fig.show()
    
    print("\n=== Feature Importance 상위 10개 ===")
    for _, row in importance_df.tail(10).iloc[::-1].iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")

elif hasattr(final_model, 'coef_'):
    # Logistic Regression 계수
    coef_df = pd.DataFrame({
        'feature': feature_cols,
        'coefficient': final_model.coef_[0]
    })
    coef_df['abs_coef'] = coef_df['coefficient'].abs()
    coef_df = coef_df.sort_values('abs_coef', ascending=True).tail(15)
    
    fig = px.bar(
        coef_df,
        x='coefficient',
        y='feature',
        orientation='h',
        title='Feature Coefficients (상위 15개)',
        color='coefficient',
        color_continuous_scale='RdBu_r'
    )
    fig.update_layout(height=500)
    fig.show()
    
    print("\n=== Feature Coefficients 상위 10개 (절대값 기준) ===")
    for _, row in coef_df.tail(10).iloc[::-1].iterrows():
        sign = '+' if row['coefficient'] >= 0 else ''
        print(f"  {row['feature']}: {sign}{row['coefficient']:.4f}")


=== Feature Coefficients 상위 10개 (절대값 기준) ===
  roe: -2.0370
  volatility_1y: +0.9466
  pb_ratio: +0.8698
  roa: +0.7464
  reinvestment_rate: +0.5068
  operating_margin_trend: -0.5053
  fcf_yield: +0.4267
  earnings_quality: -0.4174
  pe_ratio: +0.3712
  operating_margin: +0.3672


---
## 9. 저장 & 요약

In [52]:
# 모델 저장
model_path = os.path.join(model_dir, 'final_model.joblib')
joblib.dump(final_model, model_path)
print(f"✅ 모델 저장: {model_path}")

# 모델 비교 결과 저장
results_df.to_csv(os.path.join(model_dir, 'model_comparison.csv'), index=False)
print(f"✅ 모델 비교 결과 저장: {os.path.join(model_dir, 'model_comparison.csv')}")

✅ 모델 저장: /Users/kuka/5X Finder/models/final_model.joblib
✅ 모델 비교 결과 저장: /Users/kuka/5X Finder/models/model_comparison.csv


In [53]:
# 학습 로그 저장
training_log = {
    'timestamp': datetime.now().isoformat(),
    'dataset': {
        'total_samples': len(dataset),
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'train_years': train_years,
        'test_years': test_years,
        'features': len(feature_cols)
    },
    'smote': {
        'before_ratio': f"1:{(y_train == 0).sum() / (y_train == 1).sum():.1f}",
        'after_ratio': f"1:{(y_train_resampled == 0).sum() / (y_train_resampled == 1).sum():.1f}"
    },
    'best_model': {
        'name': best_model_name,
        'params': grid_search.best_params_,
        'cv_score': float(grid_search.best_score_)
    },
    'test_performance': {
        'roc_auc': float(roc_auc_score(y_test, y_final_proba)),
        'recall': float(recall_score(y_test, y_final_pred)),
        'precision': float(precision_score(y_test, y_final_pred, zero_division=0)),
        'f1': float(f1_score(y_test, y_final_pred))
    }
}

log_path = os.path.join(model_dir, 'training_log.json')
with open(log_path, 'w') as f:
    json.dump(training_log, f, indent=2)
    
print(f"✅ 학습 로그 저장: {log_path}")

✅ 학습 로그 저장: /Users/kuka/5X Finder/models/training_log.json


In [54]:
# 최종 요약
print("=" * 50)
print("📋 Model Training 완료")
print("=" * 50)

print(f"\n📊 데이터셋")
print(f"   Train: {len(X_train):,}개 ({train_years[0]}-{train_years[-1]})")
print(f"   Test: {len(X_test):,}개 ({test_years[0]}-{test_years[-1]})")
print(f"   Feature: {len(feature_cols)}개")

print(f"\n🔄 점진적 모델 학습")
print(f"   1. Logistic Regression (베이스라인)")
print(f"   2. Decision Tree (비선형)")
print(f"   3. Random Forest (앙상블 - 배깅)")
print(f"   4. Gradient Boosting (앙상블 - 부스팅)")
print(f"   5. XGBoost (개선된 부스팅)")

print(f"\n🎯 최종 모델")
print(f"   모델: {best_model_name}")
print(f"   선택 이유: Recall 기준 (5배 종목을 놓치지 않는 것이 중요)")
print(f"   최적 파라미터: {grid_search.best_params_}")

print(f"\n📈 Test 성능")
print(f"   ROC-AUC: {roc_auc_score(y_test, y_final_proba):.4f}")
print(f"   Recall: {recall_score(y_test, y_final_pred):.4f} (31개 중 {int(recall_score(y_test, y_final_pred) * 31)}개 발굴)")
print(f"   Precision: {precision_score(y_test, y_final_pred, zero_division=0):.4f}")

print(f"\n📁 저장 위치")
print(f"   {model_dir}")
print(f"   - final_model.joblib")
print(f"   - model_comparison.csv")
print(f"   - training_log.json")

📋 Model Training 완료

📊 데이터셋
   Train: 3,681개 (2010-2017)
   Test: 971개 (2018-2019)
   Feature: 26개

🔄 점진적 모델 학습
   1. Logistic Regression (베이스라인)
   2. Decision Tree (비선형)
   3. Random Forest (앙상블 - 배깅)
   4. Gradient Boosting (앙상블 - 부스팅)
   5. XGBoost (개선된 부스팅)

🎯 최종 모델
   모델: Logistic Regression
   선택 이유: Recall 기준 (5배 종목을 놓치지 않는 것이 중요)
   최적 파라미터: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}

📈 Test 성능
   ROC-AUC: 0.8716
   Recall: 0.8065 (31개 중 25개 발굴)
   Precision: 0.1106

📁 저장 위치
   /Users/kuka/5X Finder/models
   - final_model.joblib
   - model_comparison.csv
   - training_log.json
